<a href="https://colab.research.google.com/github/EduardoMoraesRitter/NLP-Chatbot/blob/main/NLP_tflearn_DNN_Chatbot_Portugu%C3%AAs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!pip install tflearn

In [3]:
import nltk
nltk.download('punkt')
nltk.download('rslp')
from nltk.stem.lancaster import LancasterStemmer
#stemmer = LancasterStemmer()
stemmer = nltk.stem.RSLPStemmer()

import numpy
import tflearn
import tensorflow
import random
import json
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!








In [4]:
import json, requests

url = 'https://raw.githubusercontent.com/EduardoMoraesRitter/NLP-Chatbot/main/intents.json'
resp = requests.get(url)
data = json.loads(resp.text)
print(data)

##OU
#with open("intents.json") as file:
#    data = json.load(file)

{'intents': [{'tag': 'saudacao', 'patterns': ['ola', 'oi', 'como vai', 'eai'], 'responses': ['Ola seja bem vindo', 'como vai?'], 'context_set': ''}, {'tag': 'despedida', 'patterns': ['xau', 'tchau', 'ate mais', 'falando'], 'responses': ['ate logo'], 'context_set': ''}]}


In [5]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)

except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)#cortar a frase me palavras
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])
    #print(data)

    #criar noss sproprio dicionario de palavras unicas
    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = [] 

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)

    print(training)
    print(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)


In [6]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





---------------------------------
Run id: LKRBP2
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 8
Validation samples: 0
--
Training Step: 1  | time: 0.127s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 8/8
--
Training Step: 2  | total loss: 0.62383 | time: 0.014s
| Adam | epoch: 002 | loss: 0.62383 - acc: 0.5114 -- iter: 8/8
--
Training Step: 3  | total loss: 0.68998 | time: 0.005s
| Adam | epoch: 003 | loss: 0.68998 - acc: 0.5028 -- iter: 8/8
--
Training Step: 4  | total loss: 0.69215 | time: 0.004s
| Adam | epoch: 004 | loss: 0.69215 - acc: 0.5009 -- ite

In [9]:
'''
try:
  model.load("model.tflearn")
except:
  model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
  model.save("model.tflearn")
'''

'\ntry:\n  model.load("model.tflearn")\nexcept:\n  model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)\n  model.save("model.tflearn")\n'

In [7]:
model.load("model.tflearn")

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [9]:
def bag_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return numpy.array(bag)

In [11]:
matrix_palavra = bag_words('oi', words)
print(matrix_palavra)

results = model.predict([matrix_palavra])
print(results)

results_index = numpy.argmax(results)
print(results_index)

tag = labels[results_index]
print(tag)

[0 0 0 0 0 1 0 0 0 0]
[[0.01946444 0.9805355 ]]
1
saudacao


In [ ]:
def chat():
    print("Falar com o bot (sair)")
    while True:
            inp = input("Voce: ")
            if inp.lower() == "sair":
                break
            
            results = model.predict([bag_words(inp, words)])
            results_index = numpy.argmax(results)
            tag = labels[results_index]
            #print(tag)

            for tg in data['intents']:
                if tg['tag'] == tag:
                    responses = tg['responses']
                
            print(random.choice(responses))

chat()

Falar com o bot (sair)
Voce: oi
como vai?
Voce: xau
ate logo
